# Building a classifier for detecting tuberculosis in chest X-rays
By Jonathan Feenstra (s1024219) and Justin Huberts (s1030368)

## Pre-processing
### Step 1: Reconstruct the Montgomery County X-ray set using an autoencoder

### Step 2: Suppress the bone structure in both datasets using PCA

## ELM classifier
Train and test a binary ELM classifier using the extension for scikit-learn. Evaluate it for both of the datasets with an ROC-curve and a confusion matrix.